In [1]:
import nltk
from nltk.corpus import udhr
import keras
from nltk import FreqDist
from gensim.models.word2vec import Word2Vec
import pandas as pd
import regex
import nltk.tokenize.casual
import tensorflow as tf

In [2]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import gc

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option
df = pd.read_csv('books_labels.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', on_bad_lines='skip')

In [4]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
spanish_df = df[df['language'] == 'SPA']['text']
spanish_list = spanish_df.to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

In [5]:
spanish_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
spanish_string = ''.join(spanish_list_clean)
spanish_list_clean = spanish_string.split('.')
spanish_list_clean = [item.strip(' ') for item in spanish_list_clean]
spanish_corpus = [nltk.tokenize.casual_tokenize(item) for item in spanish_list_clean][:1000]
print(spanish_corpus[0])

['Estas', 'son', 'las', 'palabras', 'de', 'Amós', ',', 'que', 'era', 'un', 'pastor', 'de', 'Tecoa']


In [6]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
portuguese_df = df[df['language'] == 'POR']['text']
portuguese_list = portuguese_df.to_list()

In [7]:
# portuguese_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in portuguese_list]
# print(portuguese_list_clean[:50])
for i, item in enumerate(portuguese_list):
    if type(item) == float:
        portuguese_list.pop(i)
portuguese_string = ''.join(portuguese_list)
portuguese_list = portuguese_string.split('.')
portuguese_list = [item.strip(' ') for item in portuguese_list]
portuguese_corpus = [nltk.tokenize.casual_tokenize(item) for item in portuguese_list][:1000]
print(portuguese_corpus[0])

['No', 'segundo', 'ano', 'do', 'rei', 'Dario', ',', 'no', 'sexto', 'mês', ',', 'no', 'primeiro', 'dia', 'do', 'mês', ',', 'veio', 'a', 'palavra', 'do', 'Senhor', ',', 'por', 'intermédio', 'do', 'profeta', 'Ageu', ',', 'a', 'Zorobabel', ',', 'governador', 'de', 'Judá', ',', 'filho', 'de', 'Sealtiel', ',', 'e', 'a', 'Josué', ',', 'o', 'sumo', 'sacerdote', ',', 'filho', 'de', 'Jeozadaque', ',', 'dizendo', ':', 'Assim', 'fala', 'o', 'Senhor', 'dos', 'exércitos', ',', 'dizendo', ':', 'Este', 'povo', 'diz', ':', 'Não', 'veio', 'ainda', 'o', 'tempo', ',', 'o', 'tempo', 'de', 'se', 'edificar', 'a', 'casa', 'do', 'Senhor']


In [8]:
del df
del spanish_string, portuguese_string, spanish_list, spanish_list_clean, portuguese_list
gc.collect()

0

In [9]:
# New Dataset: https://huggingface.co/datasets/versae/bibles
spanish_input_texts, spanish_target_texts = [], []
portuguese_input_texts, portuguese_target_texts = [], []
spanish_vocabulary = set()
portuguese_vocabulary = set()
start_token = '[START]'
stop_token = '[END]'
unknown_token = '[UNK]'
spanish_vocabulary.add(start_token)
spanish_vocabulary.add(stop_token)
spanish_vocabulary.add(unknown_token)
portuguese_vocabulary.add(start_token)
portuguese_vocabulary.add(stop_token)
portuguese_vocabulary.add(unknown_token)

# max_training_samples = min(25000, len(spanish_corpus) - 1, len(portuguese_corpus) - 1)

for spanish_input_text in spanish_corpus:
    spanish_target_text = [start_token] + spanish_input_text + [stop_token]
    spanish_input_texts.append(spanish_input_text)
    spanish_target_texts.append(spanish_target_text)
    for char in spanish_target_text:
        if char not in spanish_vocabulary:
            spanish_vocabulary.add(char)

for portuguese_input_text in portuguese_corpus:
    portuguese_target_text = [start_token] + portuguese_input_text + [stop_token]
    portuguese_input_texts.append(portuguese_input_text)
    portuguese_target_texts.append(portuguese_target_text)
    for char in portuguese_target_text:
        if char not in portuguese_vocabulary:
            portuguese_vocabulary.add(char)

unified_vocabulary = spanish_vocabulary + portuguese_vocabulary

print(spanish_target_texts)

[['[START]', 'Estas', 'son', 'las', 'palabras', 'de', 'Amós', ',', 'que', 'era', 'un', 'pastor', 'de', 'Tecoa', '[END]'], ['[START]', 'Tuvo', 'una', 'visión', 'acerca', 'de', 'Israel', 'durante', 'los', 'reinados', 'de', 'Uzías', 'en', 'Judá', ',', 'y', 'de', 'Jeroboán', 'hijo', 'de', 'Joás', 'en', 'Israel', '[END]'], ['[START]', 'Esto', 'sucedió', 'dos', 'años', 'antes', 'del', 'terremoto', '[END]'], ['[START]', 'le', 'prenderé', 'fuego', 'a', 'la', 'muralla', 'de', 'Tiro', ',', 'y', 'éste', 'consumirá', 'sus', 'palacios', '[END]'], ['[START]', 'Así', 'ha', 'dicho', 'el', 'Señor', ':', 'Por', 'tres', 'pecados', 'de', 'Edom', ',', 'y', 'por', 'el', 'cuarto', ',', 'no', 'revocaré', 'su', 'castigo', '[END]'], ['[START]', 'Por', 'perseguir', 'espada', 'en', 'mano', 'a', 'su', 'hermano', ',', 'violando', 'así', 'todo', 'afecto', 'natural', ',', 'y', 'por', 'robarle', 'siempre', ',', 'y', 'con', 'furor', 'guardarle', 'rencor', 'todo', 'el', 'tiempo', ',', 'le', 'prenderé', 'fuego', 'a', 'Te

In [10]:
spanish_vocabulary = sorted(spanish_vocabulary)
portuguese_vocabulary = sorted(portuguese_vocabulary)

spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
unified_vocab_size = len(unified_vocabulary)
max_spanish_seq_length = max([len(txt) for txt in spanish_target_texts])
max_portuguese_seq_length = max([len(txt) for txt in portuguese_target_texts])
max_unified_seq_length = max(max_spanish_seq_length, max_portuguese_seq_length)

spanish_token_index = dict([(token, i+1) for i, token in
                          enumerate(spanish_vocabulary)])
portuguese_token_index = dict([(token, i+1) for i, token in
                          enumerate(portuguese_vocabulary)])
unified_token_index = dict([(token, i+1) for i, token in
                          enumerate(unified_vocabulary)])
reverse_spanish_token_index = dict([(i, token) for i, token in
                          spanish_token_index.items()])
reverse_portuguese_token_index = dict([(i, token) for i, token in
                          portuguese_token_index.items()])
reverse_unified_token_index = dict([(i, token) for i, token in
                          unified_token_index.items()])

In [11]:
import numpy as np

# encoder_input_data = np.zeros((len(spanish_input_texts) + len(portuguese_input_texts),
#                                max(max_spanish_seq_length, max_portuguese_seq_length), spanish_vocab_size + portuguese_vocab_size),
#                                dtype='float32')

# Update with both
spanish_encoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')
spanish_decoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')
spanish_decoder_target_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')

portuguese_encoder_input_data = np.zeros((len(spanish_input_texts), max_portuguese_seq_length),
                               dtype='int32')
portuguese_decoder_input_data = np.zeros((len(spanish_input_texts), max_portuguese_seq_length),
                               dtype='int32')
portuguese_decoder_target_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')

for i, (input_text, target_text) in enumerate(zip(spanish_input_texts, spanish_target_texts)):
    for t, token in enumerate(input_text):
        spanish_encoder_input_data[
            i, t] = spanish_token_index[token]
    for t, token in enumerate(target_text):
        spanish_decoder_input_data[
            i, t] = spanish_token_index[token]
        if t > 0:
            spanish_decoder_target_data[i, t - 1] = spanish_token_index[token]

In [12]:
spanish_encoder_input_data

array([[ 446, 5518, 3710, ...,    0,    0,    0],
       [ 949, 5881, 6038, ...,    0,    0,    0],
       [ 449, 5557, 2619, ...,    0,    0,    0],
       ...,
       [ 760,   77,  604, ...,    0,    0,    0],
       [ 573,    8, 4876, ...,    0,    0,    0],
       [ 736, 5639, 5413, ...,    0,    0,    0]], dtype=int32)

In [13]:
batch_size = 64
epochs = 20
num_neurons = 256

In [14]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense


# spanish_encoder_inputs = Input(shape=(None, spanish_vocab_size))
# spanish_encoder = LSTM(num_neurons, return_state=True)
# spanish_encoder_outputs, s_state_h, s_state_c = spanish_encoder(spanish_encoder_inputs)
# spanish_encoder_states = (s_state_h, s_state_c)

# spanish_decoder_inputs = Input(shape=(None, spanish_vocab_size))
# spanish_decoder_lstm = LSTM(
#     num_neurons, return_sequences=True, return_state=True)
# spanish_decoder_outputs, _, _ = spanish_decoder_lstm(
#     spanish_decoder_inputs, initial_state=spanish_encoder_states)
# spanish_decoder_dense = Dense(
#     spanish_vocab_size, activation='softmax')
# spanish_decoder_outputs = spanish_decoder_dense(spanish_decoder_outputs)
# spanish_spanish_model = Model(
#     inputs=[spanish_encoder_inputs, spanish_decoder_inputs],
#     outputs=spanish_decoder_outputs)

# spanish_spanish_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#               metrics=['acc'])
# spanish_spanish_model.fit([spanish_encoder_input_data, spanish_decoder_input_data],
#           spanish_decoder_target_data, batch_size=batch_size, epochs=epochs,
#           validation_split=0.1)

In [15]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [16]:
from transformer import *
# transformer = Transformer(
#     num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     input_vocab_size=spanish_vocab_size,
#     target_vocab_size=spanish_vocab_size,
#     dropout_rate=dropout_rate)

2024-04-09 17:44:24.930913: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-09 17:44:24.930933: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-04-09 17:44:24.930938: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-04-09 17:44:24.930955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-09 17:44:24.930966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
encoder = Encoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=spanish_vocab_size,
    dropout_rate=dropout_rate)

In [17]:
#output = transformer((spanish_encoder_input_data, spanish_decoder_target_data))
#print(output.shape)

In [18]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [19]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [20]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [21]:
ds = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
#tf.data.Dataset.bucket_by_sequence_length(ds)

In [22]:
attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

AttributeError: 'DecoderLayer' object has no attribute 'last_attn_scores'

In [23]:
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
tf.config.run_functions_eagerly(True)
transformer.fit(ds.batch(64), epochs=epochs)

Epoch 1/20


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

32/32 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - loss: 8.7381 - masked_accuracy: 7.8941e-04
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - loss: 8.5962 - masked_accuracy: 0.0193
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - loss: 8.4472 - masked_accuracy: 0.0606
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 8.3139 - masked_accuracy: 0.0607
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 8.1415 - masked_accuracy: 0.0604
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 7.9229 - masked_accuracy: 0.0605
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 7.6668 - masked_accuracy: 0.0616
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 7.3867 - masked_accuracy: 0.0668
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - loss: 7.0979 - masked_accuracy: 0.0877
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - loss: 6.8151 - masked_accuracy: 0.1064
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - loss: 6.5542 - masked_accuracy: 0.1105
Epoc